### 1. Set environmental variables

In [1]:
from env_config import env_config

config = env_config()

2025-06-14 18:22:53,962 - INFO - No .env file found at /Users/drew_wilkins/Drews_Files/Drew/Python/Repositories/.env
2025-06-14 18:22:53,963 - INFO - Running in context: streamlit
2025-06-14 18:22:53,963 - INFO - Force user authentication: False
2025-06-14 18:22:54,067 - INFO - Set balance of env values from Streamlit secrets


OPTIONAL: Review environmental variables
WARNING: DO NOT COMMIT WITH THIS CELL OUTPUT SHOWING

In [ ]:
from pprint import pprint

pprint(config)

In [ ]:
import toml

secrets = toml.load(".streamlit/secrets.toml")

In [ ]:
print(secrets.keys())

In [ ]:
import streamlit as st
print("secrets keys:", list(st.secrets.keys()))
print("full secrets object:", st.secrets)

print(st.secrets.keys())

In [ ]:
import json

creds_info = json.loads(
    st.secrets["GCP_CREDENTIALS_FOR_STREAMLIT_USCGAUX_APP"])

In [ ]:
api_key = st.secrets["QDRANT_API_KEY"]

### 2. Init clients

In [2]:
from gcp_utils import get_gcp_credentials, init_sheets_client, init_drive_client
from qdrant_utils import init_qdrant_client


creds = get_gcp_credentials()
sheets_client = init_sheets_client(creds)
drive_client = init_drive_client(creds)
qdrant_client = init_qdrant_client("cloud")

2025-06-14 18:22:59,287 - INFO - No .env file found at /Users/drew_wilkins/Drews_Files/Drew/Python/Repositories/.env
2025-06-14 18:22:59,288 - INFO - Running in context: streamlit
2025-06-14 18:22:59,288 - INFO - Force user authentication: False
2025-06-14 18:22:59,288 - INFO - Set balance of env values from Streamlit secrets
2025-06-14 18:22:59,623 - INFO - No .env file found at /Users/drew_wilkins/Drews_Files/Drew/Python/Repositories/.env
2025-06-14 18:22:59,623 - INFO - Running in context: streamlit
2025-06-14 18:22:59,623 - INFO - Force user authentication: False
2025-06-14 18:22:59,623 - INFO - Set balance of env values from Streamlit secrets
2025-06-14 18:22:59,668 - INFO - ✅ Google Sheets client initialized successfully with scoped credentials.
2025-06-14 18:22:59,670 - INFO - file_cache is only supported with oauth2client<4.0.0
2025-06-14 18:22:59,671 - INFO - ✅ Google Drive client initialized successfully with scoped credentials.
2025-06-14 18:22:59,671 - INFO - Initializing Q

### 3. Test main orcehstration functions

Validate rows in LIBRARY_UNIFIED

In [ ]:
from gcp_utils import fetch_sheet_as_df
from library_utils import validate_all_rows_format
from IPython.display import display, HTML


library_df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
valid_df, invalid_df, log_df = validate_all_rows_format(library_df)

if not log_df.empty:
    print(f"\nValidation error detail:")
    display(HTML(log_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

if not invalid_df.empty:
    print(f"\nInvalid rows detail:")
    display(HTML(invalid_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))

Promote tagged

In [ ]:
from promote_files import promote_files

promote_files(drive_client, sheets_client, qdrant_client)

Delete tagged


In [ ]:
from delete_tagged import delete_tagged


deleted_rows_df = delete_tagged(drive_client, sheets_client, qdrant_client)

Create new status map

In [3]:
from status_map import build_status_map
from IPython.display import display, HTML

status_df, issues_df = build_status_map(drive_client, sheets_client, qdrant_client)
if not issues_df.empty:
    display(HTML(issues_df.to_html(
        notebook=True, max_cols=30, max_rows=2, escape=False)))

2025-06-14 18:23:03,038 - INFO - No .env file found at /Users/drew_wilkins/Drews_Files/Drew/Python/Repositories/.env
2025-06-14 18:23:03,039 - INFO - Running in context: streamlit
2025-06-14 18:23:03,039 - INFO - Force user authentication: False
2025-06-14 18:23:03,039 - INFO - Set balance of env values from Streamlit secrets
2025-06-14 18:23:04,045 - INFO - Sheet1
2025-06-14 18:23:04,428 - INFO - ✅ Fetched and converted worksheet 1glNjmKYloO0u6tCd5qy3z5ee8I-MkqwUxWsWEvLMBkM with 223 rows.
2025-06-14 18:23:06,713 - INFO - HTTP Request: POST https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections/ASK_vectorstore/points/scroll "HTTP/1.1 200 OK"
2025-06-14 18:23:10,618 - INFO - HTTP Request: POST https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.qdrant.io:6333/collections/ASK_vectorstore/points/scroll "HTTP/1.1 200 OK"
2025-06-14 18:23:14,310 - INFO - HTTP Request: POST https://0c82e035-1105-40f2-a0bd-ecc44a016f15.us-east4-0.gcp.cloud.q

,title,pdf_file_name,pdf_id,gcp_file_id,issues,empty_pdf_id_in_sheet,empty_gcp_file_id_in_sheet,empty_gcp_file_id_in_qdrant,duplicate_pdf_id_in_sheet,in_sheet,in_drive,file_name,in_qdrant,record_count,page_count,gcp_file_ids,unique_file_count,zero_record_count,file_ids_match
166,"COAST GUARD EXTERNAL AFFAIRS MANUAL, COMDTINST...",CG Public Affairs CIM_5700_13A_2021-05-01.pdf,d354afd1-95ea-5489-9df7-3abdd322d73c,,"[Empty gcp_file_id in Sheet, Empty gcp_file_id...",False,True,True,False,True,False,None,True,132,133.0,[],0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,ALAUX 010/25 Chief Director's Final Action on ...,010_25_CHIEF_DIRECTOR_S_FINAL_ACTION_ON_NATION...,af8c8931-3913-5cad-9f02-125d982cbe86,1dxDBQAcPCzxTbZPsk5AgLC0h-HEL-2g2,[Missing in Qdrant],False,False,False,False,True,True,NaN,False,0,NaN,NaN,0,False,None


In [ ]:
if not status_df.empty:
    # Keep only rows where 'issues' is a non-empty list
    status_df_issues_only = status_df[
        status_df["issues"].apply(lambda x: isinstance(x, list) and len(x) > 0)
    ]

    display(HTML(status_df_issues_only.to_html(
        notebook=True, max_cols=30, max_rows=2, escape=False)))

In [ ]:
status_df.to_csv("tests/status_map.csv", index=False)

### 4. Test helper functions

In [ ]:
from gcp_utils import fetch_sheet

sheet = fetch_sheet(sheets_client, config["LIBRARY_UNIFIED"])
cell_value = sheet.cell(2, 3).value  # Row 2, Column 3
print(cell_value)

In [ ]:
from library_utils import fetch_rows_by_status, remove_rows
from gcp_utils import get_folder_name, fetch_sheet_as_df, file_exists
import pandas as pd

library_df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
display(library_df.head())

In [ ]:
config = env_config()

In [ ]:
import logging
from qdrant_client import QdrantClient
from qdrant_client.http import models
from typing import Any
from library_utils import fetch_sheet_as_df
from IPython.display import display, HTML
import pandas as pd
from env_config import rag_config


def update_qdrant_file_ids_for_live_rows(qdrant_client: QdrantClient, sheets_client: Any, collection_name: str) -> pd.DataFrame:
    """
    For each status='live' row in LIBRARY_UNIFIED, update all Qdrant points with that pdf_id
    to have the corresponding gcp_file_id in their metadata.

    Args:
        qdrant_client: QdrantClient instance
        sheets_client: Authenticated Google Sheets client
        collection_name: Qdrant collection name

    Returns:
        pd.DataFrame of modified rows from LIBRARY_UNIFIED
    """
    # Load LIBRARY_UNIFIED and filter for status='live'
    df = fetch_sheet_as_df(sheets_client, config["LIBRARY_UNIFIED"])
    live_df = df[df["status"] == "live"].copy()
    if live_df.empty:
        logging.warning("No rows with status='live' found in LIBRARY_UNIFIED.")
        return live_df

    updated = []
    for _, row in live_df.iterrows():
        pdf_id = str(row.get("pdf_id", "")).strip()
        gcp_file_id = str(row.get("gcp_file_id", "")).strip()

        if not pdf_id or not gcp_file_id:
            continue

        filter_condition = models.Filter(
            must=[models.FieldCondition(
                key="metadata.pdf_id", match=models.MatchValue(value=pdf_id))]
        )

        scroll_offset = None
        while True:
            points, scroll_offset = qdrant_client.scroll(
                collection_name=collection_name,
                scroll_filter=filter_condition,
                with_payload=True,
                with_vectors=False,
                limit=100,
                offset=scroll_offset,
            )
            if not points:
                break

            for point in points:
                point_id = point.id
                metadata = point.payload.get("metadata", {})
                if metadata.get("gcp_file_id") != gcp_file_id:
                    metadata["gcp_file_id"] = gcp_file_id
                    qdrant_client.set_payload(
                        collection_name=collection_name,
                        payload={"metadata": metadata},
                        points=[point_id],
                    )
                    updated.append({
                        "pdf_id": pdf_id,
                        "point_id": point_id,
                        "new_gcp_file_id": gcp_file_id
                    })

            if scroll_offset is None:
                break

    if updated:
        logging.info(
            f"Updated {len(updated)} Qdrant points with correct gcp_file_id.")
    else:
        logging.info("No updates were necessary.")

    return pd.DataFrame(updated)


updated_df = update_qdrant_file_ids_for_live_rows(
    qdrant_client, sheets_client, rag_config("qdrant_collection_name"))

if not updated_df.empty:
    display(HTML(summaries_df.to_html(notebook=True,
            max_cols=30, max_rows=10, escape=False)))